# Exploratory data analysis for Crime Locations in Auckland
A Capstone Project - The Battle of Neighborhoods

## Introduction

Auckland(Tāmaki Makaurau) is the biggest city in New Zealand, with the population growing, the crimes happening frequency is increasing to, This project is 

## Table of Contents
1. Data Preparation

## 1 Data Preparation

### 1.1 Prepare Python Dependecies

In [108]:
!conda install -c conda-forge geopy matplotlib python-dotenv folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /Users/tim/miniconda3/envs/data

  added / updated specs:
    - folium=0.5.0
    - geopy
    - matplotlib
    - python-dotenv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python-dotenv-0.10.5       |             py_0          18 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          18 KB

The following NEW packages will be INSTALLED:

  python-dotenv      conda-forge/noarch::python-dotenv-0.10.5-py_0



python-dotenv-0.10.5 | 18 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


### 1.2 Prepare Crimes Data

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('./NZ_Crimes.csv')

In [16]:
df_akl = df[df['Territorial Authority']=='Auckland.']

columns=[
    'ANZSOC Division',
    'ANZSOC Group',
    'ANZSOC Subdivision',
    'Area Unit',
    'Locn Type Division',
    'Meshblock',
    'Occurrence Day Of Week',
    'Occurrence Hour Of Day',
    'Weapon',
    ' Month Year',
]

df_akl=df_akl[columns].astype({
    'Meshblock': 'int32',
    'Occurrence Hour Of Day': 'int32',
})

# formate the date (Month Year), using date instead of string
from datetime import datetime
df_akl[' Month Year']=df_akl[' Month Year'].apply(
    lambda x: datetime.strptime(x, '%B %Y')
)

df_akl = df_akl.rename(columns={
    ' Month Year': 'YearMonth'
})

df_akl.head()

,ANZSOC Division,ANZSOC Group,ANZSOC Subdivision,Area Unit,Locn Type Division,Meshblock,Occurrence Day Of Week,Occurrence Hour Of Day,Weapon,YearMonth
24,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Albany.,Other Location,178900,Saturday,15,Not Applicable,2019-01-01
25,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433501,Saturday,15,Not Applicable,2019-01-01
26,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433600,Saturday,15,Not Applicable,2019-01-01
27,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,434000,Saturday,15,Not Applicable,2019-01-01
28,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Mt Wellington South.,Other Location,637400,Saturday,15,Not Applicable,2019-01-01


In [17]:
df_akl.dtypes

ANZSOC Division                   object
ANZSOC Group                      object
ANZSOC Subdivision                object
Area Unit                         object
Locn Type Division                object
Meshblock                          int32
Occurrence Day Of Week            object
Occurrence Hour Of Day             int32
Weapon                            object
YearMonth                 datetime64[ns]
dtype: object

In [102]:
rows_num = df_akl.shape[0]
print('crimed date is loaded and prepared, totally {} rows'.format(rows_num))

crimed date is loaded and prepared, totally 69725 rows


### 1.2 Prepare Location Data

the way to get the latitude and longitude for each suburbs (area names)
```
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="akl_spider")
location = geolocator.geocode("{} auckland new zealand".format('albany'))
```
the way generated the `area_location.json` notebook.
```
with open('area_location.json', 'w') as file:
    json.dump(area_dict, file)
```

loading the generated json file instead of run the geocode each time will increase notebook speadd siginifitantly.


In [167]:
# this json file contains suburbs of Auckland
## which is generated through calling geopy geocode function
## it is too heavy to call it every time to run the notebook, so the json file is generated
# some one which cannot get through 'geocoder' which will be set to (0,0)
# for example `Inlet-Waitemata Harbour` which is not a correct suburb in Auckland, this kind of rows will be droped in this step
areas = list(df_akl['Area Unit'].unique())
locations_2020 = {}
with open('locations_2020.json', 'r') as file:
    locations_2020=json.load(file)

In [169]:
if (len(locations_2020.keys()) < len(areas)):
    # the json file loades seems like outdated
    from geopy.geocoders import Nominatim
    import time
    locations = []

    def get_location(area_name, ua="Mozilla/5.0 (X11; Linux i686; rv:67.0) Gecko/20100101 Firefox/67.0"):
        geolocator = Nominatim(user_agent=ua)
        location = geolocator.geocode("{} auckland new zealand".format(area_name))
        time.sleep(1)
        return location

    def get_locations(area_names):
        return list(map(get_location, area_names))

    # runing this too fast will cause time out error
    # better to run the codes below in separate cells manuelly
    locations += get_locations(areas[:50])
    locations += get_locations(areas[50:100])
    locations += get_locations(areas[100:180])
    locations += get_locations(areas[180:260])
    locations += get_locations(areas[260:350])
    locations += get_locations(areas[350:400])
    locations += get_locations(areas[400:])
    locations_2020 = locations
else:
    print('locations is imported through loading json file')

locations is imported through loading json file


In [171]:
# a simple test to make sure the json is loaded correctly
print('json loading successfully ' if 'Albany.' in location_dict.keys() and len(locations) >= len(areas) else '') 

json loading successfully 


In [173]:
df_akl['latitude'] = df_akl['Area Unit'].apply(lambda x: location_dict[x][0])
df_akl['longitude'] = df_akl['Area Unit'].apply(lambda x: location_dict[x][1])

df_akl = df_akl[df_akl['latitude']!=0]

In [174]:
rows_num_after_drop = df_akl.shape[0]

In [175]:
df_akl.head()

,ANZSOC Division,ANZSOC Group,ANZSOC Subdivision,Area Unit,Locn Type Division,Meshblock,Occurrence Day Of Week,Occurrence Hour Of Day,Weapon,YearMonth,latitude,longitude
24,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Albany.,Other Location,178900,Saturday,15,Not Applicable,2019-01-01,-36.727058,174.698055
25,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433501,Saturday,15,Not Applicable,2019-01-01,-36.848911,174.765226
26,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433600,Saturday,15,Not Applicable,2019-01-01,-36.848911,174.765226
27,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,434000,Saturday,15,Not Applicable,2019-01-01,-36.848911,174.765226
28,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Mt Wellington South.,Other Location,637400,Saturday,15,Not Applicable,2019-01-01,-36.904793,174.835118


In [176]:
print('there {} rows which does not have a clean area name'.format(rows_num - rows_num_after_drop))
print('now we have {} rows in the table for the next step'.format(rows_num_after_drop))

there 218 rows which does not have a clean area name
now we have 69507 rows in the table for the next step


### 1.3 Prepare Venue Data 
calling foursquare api to get the related venues for each suburbs

In [177]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


**Please set up the `.env` firstly** before running the cell below.

In [182]:
import os
import requests
# define a function to get the foursquare result
def get_venue(lat, lng, radius, limit):
    client_id = os.environ['CLIENT_ID']
    client_secret=os.environ['CLIENT_SECRET']
    version=os.environ['VERSION']
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        client_id, 
        client_secret, 
        version, 
        lat, 
        lng, 
        radius, 
        limit)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    return results
get_venue(-36.727, 174.698, 500, 10)

# try use foursquare client package instead

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=4UOVAM4NYGTOHYNY51HMDI5YJD2M1TNDS04U2ONBVRNANLAK&client_secret=1O3BNCGK4IQMIRVE42MHEIKEX3MSSRVGHVK2AQFHKTNLPDTF&v=20180605&ll=-36.727,174.698&radius=500&limit=10 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1231e8d50>: Failed to establish a new connection: [Errno 61] Connection refused'))

try use the `get_venue` to get result of `albany` (-36.727058 	174.698055)

In [181]:
albany_lat_lng = (-36.727058, 174.698055)
get_venue(albany_lat_lng[0], albany_lat_lng[1], 500, 100)

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=4UOVAM4NYGTOHYNY51HMDI5YJD2M1TNDS04U2ONBVRNANLAK&client_secret=1O3BNCGK4IQMIRVE42MHEIKEX3MSSRVGHVK2AQFHKTNLPDTF&v=20180605&ll=-36.727058,174.698055&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1230b9550>: Failed to establish a new connection: [Errno 61] Connection refused'))